# BLOOM-560M on SageMaker using Hugging Face Text Generation Inference (TGI) DLC

This notebook demonstrates how to deploy BLOOM-560M Language Model using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.
BigScience Large Open-science Open-access Multilingual (BLOOM) language model from BigScience is a multi-lingual LLM supporting text generation in 46 natural languages and 13 programming languages.

TGI is an open source, high performance inference library that can be used to deploy large language models from Hugging Face’s repository in minutes. The library includes advanced functionality like model parallelism and dynamic batching to simplify production inference with large language models like flan-t5-xxl, LLaMa, StableLM, and GPT-NeoX.


## Setup

Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [ ]:
%pip install pip==8.1.1 --quiet #Added due to intermittent distutils package error - optional if no error
%pip install "sagemaker>=2.163.0" --quiet

## Setup account and role

Then, we import the SageMaker python SDK and instantiate a sagemaker_session which we use to determine the current region and execution role.

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

print(sagemaker.__version__)

## Retrieve the LLM Image URI

We use the helper function get_huggingface_llm_image_uri() to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter backend and several optional parameters. The backend specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [ ]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)

## Create the Hugging Face Model

Next we configure the model object by specifying a unique name, the image_uri for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the HF_MODEL_ID which corresponds to the model from the HuggingFace Hub that will be deployed, and the HF_TASK which configures the inference task to be performed by the model.

You should also define SM_NUM_GPUS, which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here, you should set SM_NUM_GPUS to the number of available GPUs on your selected instance type. For example, in this tutorial, we set SM_NUM_GPUS to 4 because our selected instance type ml.g5.12xlarge has 4 available GPUs.

In [ ]:
model_name = "bloom-560m-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'bigscience/bloom-560m',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'4'
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)

## Creating a SageMaker Endpoint
Next we deploy the model by invoking the deploy() function. Here we use an ml.g5.12xlarge instance which come with 4 NVIDIA A10 GPUs. By setting the SM_NUM_GPUS environment variable to 4 in the last code block, we indicate that this model should be sharded across all 4 GPU devices.

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.12xlarge",
    endpoint_name=model_name
)

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the predict() function.

In [ ]:
input_data = {
  "inputs": "What is the weather forecast for Seattle today?",
  "parameters": {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "watermark": True
  }
}

response = predictor.predict(input_data)
print(response[0]["generated_text"])

## Create sample chatbot application backed by SageMaker (Optional)

In [ ]:
%pip install --upgrade pip --quiet
%pip install gradio --upgrade --quiet

In [ ]:
import gradio as gr

# hyperparameters for llm
parameters = {
    "do_sample": True,
    "top_p": 0.7,
    "temperature": 0.7,
    "top_k": 50,
    "max_new_tokens": 256,
    "repetition_penalty": 1.03,
    "stop": ["<|endoftext|>"]
  }

with gr.Blocks() as demo:
    gr.Markdown("## Chat with BLOOM-560M LLM using Amazon SageMaker")
    with gr.Column():
        chatbot = gr.Chatbot()
        with gr.Row():
            with gr.Column():
                message = gr.Textbox(label="Chat Message Box", placeholder="Chat Message Box", show_label=False)
            with gr.Column():
                with gr.Row():
                    submit = gr.Button("Submit")
                    clear = gr.Button("Clear")

    def respond(message, chat_history):
        # convert chat history to prompt
        converted_chat_history = ""
        if len(chat_history) > 0:
          for c in chat_history:
            converted_chat_history += f"<|prompter|>{c[0]}<|endoftext|><|assistant|>{c[1]}<|endoftext|>"
        prompt = f"{converted_chat_history}<|prompter|>{message}<|endoftext|><|assistant|>"

        # send request to endpoint
        llm_response = predictor.predict({"inputs": prompt, "parameters": parameters})

        # remove prompt from response
        parsed_response = llm_response[0]["generated_text"][len(prompt):]
        chat_history.append((message, parsed_response))
        return "", chat_history

    submit.click(respond, [message, chatbot], [message, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=True)

We can use this URL hosting the sample application to perform text generation with the model.

## Cleaning Up

After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion

In this tutorial, we used a TGI container to deploy BLOOM-560m using 4 GPUs on a SageMaker ml.g5.12xlarge instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.